## TUTORIAL 05 - Discrete Empirical Interpolation Method for non-affine elliptic problems
**_Keywords: discrete empirical interpolation method_**

### 1. Introduction
In this Tutorial, we consider steady heat conduction in a two-dimensional square domain $\Omega = (-1, 1)^2$.
The boundary $\partial\Omega$ is kept at a reference temperature (say, zero). The conductivity coefficient is fixed to 1, while the heat source is characterized by the following expression
$$
g(\boldsymbol{x}; \boldsymbol{\mu}) = \exp\{ -2 (x_0-\mu_0)^2 - 2 (x_1 - \mu_1)^2\} \quad \forall \boldsymbol{x} = (x_0, x_1) \in \Omega.
$$

The parameter vector $\boldsymbol{\mu}$, given by 
$$
\boldsymbol{\mu} = (\mu_0,\mu_1)
$$
affects the center of the Gaussian source $g(\boldsymbol{x}; \boldsymbol{\mu})$, which could be located at any point $\Omega$. Thus, the parameter domain is
$$
\mathbb{P}=[-1,1]^2.
$$

In order to obtain a faster evaluation (yet, provably accurate) of the problem we propose to use a certified reduced basis approximation for the problem. In order to preserve the affinity assumption (for the sake of performance) the discrete empirical interpolation method will be used on the forcing term $g(\boldsymbol{x}; \boldsymbol{\mu})$.

### 2. Parametrized formulation

Let $u(\boldsymbol{\mu})$ be the temperature in the domain $\Omega$.

We will directly provide a weak formulation for this problem: for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})\in\mathbb{V}$ such that

$$a\left(u(\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)=f(v;\boldsymbol{\mu})\quad \forall v\in\mathbb{V}$$

where

* the function space $\mathbb{V}$ is defined as
$$
\mathbb{V} = \left\{ v \in H^1(\Omega(\mu_0)): v|_{\partial\Omega} = 0\right\}
$$
Note that, as in the previous tutorial, the function space is parameter dependent due to the shape variation. 
* the parametrized bilinear form $a(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$a(u,v;\boldsymbol{\mu}) = \int_{\Omega} \nabla u \cdot \nabla v \ d\boldsymbol{x}$$
* the parametrized linear form $f(\cdot; \boldsymbol{\mu}): \mathbb{V} \to \mathbb{R}$ is defined by
$$f(v;\boldsymbol{\mu}) = \int_\Omega g(\boldsymbol{\mu}) v  \ d\boldsymbol{x}.$$

In [ ]:
import os
import sys
sys.path.append('../../')

from mlnics import NN, Losses, Normalization, RONNData, IO, Training, ErrorAnalysis
from dolfin import *
from rbnics import *
import torch
import numpy as np

torch.manual_seed(42)
np.random.seed(42)

## 3. Affine decomposition

The parametrized bilinear form $a(\cdot, \cdot; \boldsymbol{\mu})$ is trivially affine.
The discrete empirical interpolation method will be used on the forcing term $g(\boldsymbol{x}; \boldsymbol{\mu})$ to obtain an efficient (approximately affine) expansion of $f(\cdot; \boldsymbol{\mu})$.

In [ ]:
@DEIM("online", basis_generation="Greedy")
@ExactParametrizedFunctions("offline")
class Gaussian(EllipticCoerciveProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        EllipticCoerciveProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.u = TrialFunction(V)
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data=subdomains)
        self.f = ParametrizedExpression(
            self, "exp(- 2 * pow(x[0] - mu[0], 2) - 2 * pow(x[1] - mu[1], 2))", mu=(0., 0.),
            element=V.ufl_element())
        # note that we cannot use self.mu in the initialization of self.f, because self.mu has not been initialized yet

    # Return custom problem name
    def name(self):
        return "GaussianDEIM"

    # Return the alpha_lower bound.
    def get_stability_factor_lower_bound(self):
        return 1.

    # Return theta multiplicative terms of the affine expansion of the problem.
    def compute_theta(self, term):
        if term == "a":
            return (1.,)
        elif term == "f":
            return (1.,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    def assemble_operator(self, term):
        v = self.v
        dx = self.dx
        if term == "a":
            u = self.u
            a0 = inner(grad(u), grad(v)) * dx
            return (a0,)
        elif term == "f":
            f = self.f
            f0 = f * v * dx
            return (f0,)
        elif term == "dirichlet_bc":
            bc0 = [DirichletBC(self.V, Constant(0.0), self.boundaries, 1),
                   DirichletBC(self.V, Constant(0.0), self.boundaries, 2),
                   DirichletBC(self.V, Constant(0.0), self.boundaries, 3)]
            return (bc0,)
        elif term == "inner_product":
            u = self.u
            x0 = inner(grad(u), grad(v)) * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")

## 4. Main program
### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [ ]:
mesh = Mesh("data/gaussian.xml")
subdomains = MeshFunction("size_t", mesh, "data/gaussian_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/gaussian_facet_region.xml")

### 4.2. Create Finite Element space (Lagrange P1)

In [ ]:
V = FunctionSpace(mesh, "Lagrange", 1)

### 4.3. Allocate an object of the Gaussian class

In [ ]:
problem = Gaussian(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(-1.0, 1.0), (-1.0, 1.0)]
problem.set_mu_range(mu_range)

### 4.4. Prepare reduction with a reduced basis method

In [ ]:
reduction_method = ReducedBasis(problem)
reduction_method.set_Nmax(20, DEIM=21)
reduction_method.set_tolerance(1e-4, DEIM=1e-8)

### 4.5. Perform the offline phase

#### 4.5.1 Fit Reduction Method

In [ ]:
reduction_method.initialize_training_set(100, DEIM=60)
reduced_problem = reduction_method.offline()

In [ ]:
class Net(nn.Module):
    def __init__(self, in_size, out_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_size, 60)
        self.fc2 = nn.Linear(60, 60)
        self.fc3 = nn.Linear(60, out_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.fc2(x)
        x = torch.tanh(x)
        x = self.fc3(x)
        return x

In [ ]:
# Step 1
# Get nonlinear_terms, parameters, N0
nonlinear_terms = np.zeros((len(reduction_method.training_set), len(reduction_method.training_set), reduced_problem.N))
params = np.array(reduction_method.training_set)
N0 = lambda sol, param_idx: 0 # N0 identically 0

solutions = []
Basis_Matrix = np.array([v.vector()[:] for v in reduced_problem.basis_functions])

for mu in reduction_method.training_set:
    problem.set_mu(mu)
    solution = problem.solve()
    solutions.append(np.array(problem._solution.vector()[:]))

for i, mu in enumerate(reduction_method.training_set):
    problem.set_mu(mu)
    operator_form = problem.assemble_operator('f')[0]
    theta = problem.compute_theta('f')
    
    for j, solution in enumerate(solutions):
        problem._solution.vector()[:] = solution
        nonlinear_terms[i, j] = (theta * Basis_Matrix @ np.array(assemble(operator_form)[:]).reshape(-1, 1)).reshape(-1)
        
solutions_ = []
for sol in solutions:
    F = Function(V)
    F.vector()[:] = sol
    solutions_.append(np.array(reduced_problem.project(F).vector()[:]))
solutions = solutions_

In [ ]:
chosen_parameter_indices = []
fixed_mu_networks = []

# Step 2
# 2a. Set mu_1
errors = np.zeros(params.shape[0])
for i, mu_i in enumerate(params):
    # compute error
    s = 0
    for j, mu_j in enumerate(params):
        s += np.sum((nonlinear_terms[i, j] - N0(solutions[j], i))**2)
    errors[i] = s / params.shape[0]

mu_1_idx = np.argmax(errors)
mu_1 = params[mu_1_idx]
chosen_parameter_indices.append(mu_1_idx)
print("max error:", np.max(errors))
print("mu_1 index:", mu_1_idx)

# 2b. Train Network_{mu_1}(u) to approximate Nonlinearity(u; mu_1)
print("\nTraining network to approximate nonlinearity...")
Network_mu_1 = Net(reduced_problem.N, reduced_problem.N)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(Network_mu_1.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9999)

x_normalization = Normalization.MinMaxNormalization(input_normalization=True)
y_normalization = Normalization.MinMaxNormalization()

x_data = x_normalization(torch.tensor(np.array(solutions)).float())
y_data = y_normalization(torch.tensor(nonlinear_terms[mu_1_idx] / np.linalg.norm(nonlinear_terms[mu_1_idx])).float().T).T



for epoch in range(30000):
    optimizer.zero_grad()
    output = Network_mu_1(x_data)
    loss = criterion(output, y_data)
    if epoch % 100 == 0:
        print(epoch, loss.item())
    loss.backward()
    optimizer.step()
    scheduler.step()

Network_mu_1.eval()
fixed_mu_networks.append(Network_mu_1)

# 2c. Find theta_1_1(mu)
print("\nFinding theta...")
thetas = np.zeros((params.shape[0], 1))
for i, mu_i in enumerate(params):
    numerator = 0
    denominator = 0
    for j, mu_j in enumerate(params):
        net_u_mu = y_normalization(Network_mu_1(x_data[j].view(1, -1)).T, normalize=False).T.detach().numpy().reshape(-1)
        numerator += np.dot(nonlinear_terms[i, j], net_u_mu)
        denominator += np.dot(net_u_mu, net_u_mu)
        
    theta_1_1_i = numerator / denominator
    thetas[i] = theta_1_1_i
    print(thetas[i])

In [ ]:
matrices = []
maximum_error = np.max(errors)
y_normalization_list = [y_normalization]
mean_errors = 1

mean_errors_list = []
mean_errors_list_all = []

#while np.mean(mean_errors) > 0.0005:
for iteration in range(7):
    # Step 3
    # 3a. Set mu_2
    N1 = lambda sol, param_idx: sum([
        thetas[param_idx][i] * y_normalization_list[i](
            net(x_normalization(torch.tensor(sol).float().view(1, -1))).T, normalize=False
        ).T.detach().numpy().reshape(-1)\
        for i, net in enumerate(fixed_mu_networks)
    ])
    
    

    errors = np.zeros(params.shape[0])
    for i, mu_i in enumerate(params):
        # compute error
        s = 0
        for j, mu_j in enumerate(params):
            s += np.sum((nonlinear_terms[i, j] - N1(solutions[j], i))**2)
        errors[i] = s / params.shape[0]
        
    mean_errors = np.zeros(params.shape[0])
    for i, mu_i in enumerate(params):
        mean_errors[i] = np.linalg.norm(nonlinear_terms[i, i] - N1(solutions[i], i)) / np.linalg.norm(nonlinear_terms[i, i])
    
    mean_errors_all = np.zeros(params.shape[0])
    for i, mu_i in enumerate(params):
        for j, mu_j in enumerate(params):
            mean_errors_all[i] += np.linalg.norm(nonlinear_terms[i, j] - N1(solutions[j], i)) / np.linalg.norm(nonlinear_terms[i, j])
        mean_errors_all[i] /= params.shape[0]
        
    print("max error:", np.max(errors))
    errors[np.array(chosen_parameter_indices)] = -1 # don't choose already chosen parameters again
    mu_2_idx = np.argmax(errors)
    mu_2 = params[mu_1_idx]
    chosen_parameter_indices.append(mu_2_idx)
    
    maximum_error = np.max(errors)
    print("mean error:", np.mean(mean_errors))
    mean_errors_list.append(np.mean(mean_errors))
    print("mean error all:", np.mean(mean_errors_all))
    mean_errors_list_all.append(np.mean(mean_errors_all))
    print("mu_2 index:", mu_2_idx)

    # 3b. Train Network_{mu_2}(u) to approximate Nonlinearity(u; mu_2)
    print("\nTraining network to approximate nonlinearity...")
    Network_mu_2 = Net(reduced_problem.N, reduced_problem.N)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(Network_mu_2.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9999)
    
    
    # need to do Gram-Schmidt on this matrix
    y_data = nonlinear_terms[mu_2_idx]
    
    for net in fixed_mu_networks:
        # form matrix of evaluations for this network
        previous_net_matrix = np.zeros((params.shape[0], nonlinear_terms.shape[2]))
        for i, mu_i in enumerate(params):
            previous_net_matrix[i] = y_normalization(net(x_data[i].view(1, -1)).T, normalize=False).T.detach().numpy().reshape(-1)
        
        # subtract out projection of y_data onto previous_net_matrix from y_data
        y_data -= np.sum(y_data * previous_net_matrix) / np.linalg.norm(previous_net_matrix) * previous_net_matrix
    
    y_data = torch.tensor(y_data / np.linalg.norm(y_data)).float()
    y_normalization = Normalization.MinMaxNormalization()
    y_data = y_normalization(y_data.T).T
    
    

    for epoch in range(30000):
        optimizer.zero_grad()
        output = Network_mu_2(x_data)
        loss = criterion(output, y_data)
        if epoch % 100 == 0:
            print(epoch, loss.item())
        loss.backward()
        optimizer.step()
        scheduler.step()

    Network_mu_2.eval()
    fixed_mu_networks.append(Network_mu_2)
    y_normalization_list.append(y_normalization)

    # 3c. Find theta_1_2(mu), theta_2_2(mu)
    print("\nFinding theta...")
    num_nets = len(fixed_mu_networks)
    thetas = np.zeros((params.shape[0], num_nets))
    for i, mu_i in enumerate(params):
        LHS_numerator = np.zeros((num_nets, num_nets))
        LHS_denominator = np.zeros((num_nets, num_nets))
        RHS_numerator = np.zeros((num_nets, 1))
        RHS_denominator = np.zeros((num_nets, 1))

        for j, mu_j in enumerate(params):
            nets_u_mu = [y_normalization_list[i_net](net(x_data[j].view(1, -1)).T, normalize=False).T.detach().numpy().reshape(-1) for i_net, net in enumerate(fixed_mu_networks)]

            for k1 in range(num_nets):
                RHS_numerator[k1] += np.dot(nonlinear_terms[i, j], nets_u_mu[k1])
                RHS_denominator[k1] += np.dot(nets_u_mu[k1], nets_u_mu[k1])
                for k2 in range(num_nets):
                    
                    LHS_numerator[k1, k2] += np.dot(nets_u_mu[k1], nets_u_mu[k2])
                    LHS_denominator[k1, k2] += np.dot(nets_u_mu[k1], nets_u_mu[k1])



        LHS = LHS_numerator / LHS_denominator
        RHS = RHS_numerator / RHS_denominator
        matrices.append(LHS)
        thetas[i] = np.linalg.solve(LHS, RHS).reshape(-1)
        #print(thetas[i])

In [ ]:
def NonlinearityApprox(u, mu):
    #thetas_ = theta_normalization(theta_net(torch.tensor(mu).float().view(1, -1)).detach().T, normalize=False).T.numpy().reshape(-1, 1)
    thetas_ = interpolate.griddata(params, thetas, mu, method='cubic').reshape(-1, 1)
    if True in np.isnan(thetas_):
        thetas_ = interpolate.griddata(params, thetas, mu, method='nearest').reshape(-1, 1)
    net_evals = np.array([y_normalization_list[i](net(x_normalization(torch.tensor(u).float().view(1, -1))).T, normalize=False).T.detach().numpy().reshape(-1) for i, net in enumerate(fixed_mu_networks)])
    return np.sum(thetas_ * net_evals, axis=0)

def NonlinearityApprox2(u, mu):
    #thetas_ = theta_normalization(theta_net(torch.tensor(mu).float().view(1, -1)).T, normalize=False).T.view(-1)
    thetas_ = interpolate.griddata(params, thetas, mu, method='cubic').reshape(-1)
    if True in np.isnan(thetas_):
        thetas_ = interpolate.griddata(params, thetas, mu, method='nearest').reshape(-1)
    s = 0
    for i, net in enumerate(fixed_mu_networks):
        s += thetas_[i] * y_normalization_list[i](net(x_normalization(u.view(1, -1))).T, normalize=False).T.view(-1)
    return s

In [ ]:
from rbnics.backends.basic.wrapping.delayed_transpose import DelayedTranspose
from rbnics.backends.online import OnlineFunction, OnlineVector
from rbnics.backends.common.time_series import TimeSeries
from rbnics.backends.dolfin.parametrized_tensor_factory import ParametrizedTensorFactory
from rbnics.backends.dolfin.evaluate import evaluate

class Approx_PINN_Loss(Losses.RONN_Loss_Base):
    """
    PINN_Loss

    ronn: object of type RONN

    RETURNS: loss function loss_fn(parameters, reduced order coefficients)
    """
    def __init__(self, ronn, normalization=None, beta=1., mu=None):
        super(Approx_PINN_Loss, self).__init__(ronn, mu)
        self.operators = None
        self.proj_snapshots = None
        self.T0_idx = None
        self.normalization = normalization
        if self.normalization is None:
            self.normalization = IdentityNormalization()

        self.beta = beta

        # if time dependent, we need the neural net to compute time derivative
        self.time_dependent = False

    def name(self):
        return "Approx_PINN"

    def _compute_operators(self):
        self.operators_initialized = True

        #self.operators = self.ronn.get_operator_matrices(self.mu)
        self.operators = self.ronn.get_reduced_operator_matrices(self.mu)

        if not self.normalization.initialized:
            self.normalization(self.ronn.get_projected_snapshots())

    def set_mu(self, mu):
        self.mu = mu
        self.operators_initialized = False

    def __call__(self, **kwargs):
        pred = kwargs["prediction_no_snap"]
        if not self.operators_initialized:
            self._compute_operators()

        pred = self.normalization(pred.T, normalize=False).T

        ##### 1st equation in system #####
        res1 = 0.0

        # these two could be combined when both not None
        if 'f' in self.operators:
            res1 -= self.operators['f']
        if 'c' in self.operators:
            self.operators['c'] = torch.zeros_like(self.operators['c'])
            for i, mu in enumerate(kwargs["input_normalization"](kwargs["normalized_mu"], normalize=False)):                
                mu = np.array(mu)
                sol = pred[i].float()
                C = NonlinearityApprox2(sol, mu).view(-1, 1).double()#.detach()
                self.operators['c'][i] = C[None, :, :]

            res1 += self.operators['c']
        
        if 'a' in self.operators:
            res1 += torch.matmul(self.operators['a'], pred[:, :, None].double())

        loss1 = torch.mean(torch.sum(res1**2, dim=1)) if type(res1) is not float else res1
        if self.ronn.problem.dirichlet_bc_are_homogeneous:
            boundary_condition_loss = 0
        else:
            boundary_condition_loss = torch.mean((pred[:, 0] - 1.)**2)

        self.value = loss1 + self.beta*boundary_condition_loss

        return self.value

    def reinitialize(self, mu):
        normalization = self.normalization
        beta = self.beta
        return Approx_PINN_Loss(self.ronn, normalization, beta, mu)

#### 4.5.2 Train PINN

Given a training set $X_{PINN} = (\boldsymbol{\mu}^{(1)}, \dots, \boldsymbol{\mu}^{(n)})$ of parameters for the PDE, we train a Physics-Informed Neural Network (PINN) $\operatorname{N}_W(\boldsymbol{\mu})$ dependent on the weights and biases $W$ of the network to minimize the loss function

$$L_{PINN}(X_{PINN}; W) = \frac1n \sum_{i=1}^n \|A(\boldsymbol{\mu^{(i)}}) \operatorname{N}_W(\boldsymbol{\mu}^{(i)}) - \boldsymbol{f}(\boldsymbol{\mu}^{(i)})\|_2^2$$

over $W$, where for a given $\boldsymbol{\mu}$, $A(\boldsymbol{\mu})$ is the assembled matrix corresponding to the pull back of $a_o$ and $\boldsymbol{f}(\boldsymbol{\mu})$ is the assembled vector corresponding to the pull back of $f_o$.

In [ ]:
# build matrices for computing DEIM
selected_indices = sorted([idx[0] for idx in problem.DEIM_approximations['f'][0].interpolation_locations.get_dofs_list()])
U = np.array(problem._assemble_operator_DEIM('f')).T
P = []
for idx in selected_indices:
    new_column = np.zeros(U.shape[0])
    new_column[idx] = 1
    P.append(new_column)
P = np.array(P).T
PtUinv = np.linalg.inv(P.T @ U)

In [ ]:
# Note that this is f, not c! Code in MLniCS probably needs to change...
selected_indices = sorted([idx[0] for idx in problem.DEIM_approximations['f'][0].interpolation_locations.get_dofs_list()])
dofs_locs = torch.tensor(V.tabulate_dof_coordinates()[selected_indices])
dx_test = torch.tensor(np.array(assemble(problem.v*dx))[selected_indices].reshape(-1, 1))

def DEIM_nonlinearity(mu):
    """
    mu: torch.tensor with shape (number of samples, number of parameters)
    """
    res = torch.exp(-2 * ((dofs_locs[:, 0].reshape(-1, 1) - mu[:, 0].view(1, -1))**2 + (dofs_locs[:, 1].reshape(-1, 1) - mu[:, 1].view(1, -1))**2))
    return torch.matmul(torch.tensor(PtUinv[:, :]), dx_test * res)

In [ ]:
input_normalization_pinn = Normalization.StandardNormalization(input_normalization=True)
output_normalization_pinn = Normalization.StandardNormalization()

pinn_net  = NN.RONN("PINN", problem, reduction_method, n_hidden=2, n_neurons=40)
pinn_loss = Losses.PINN_Loss(pinn_net, output_normalization_pinn, DEIM_func_f=DEIM_nonlinearity)
data      = RONNData.RONNDataLoader(pinn_net, validation_proportion=0.2, 
                                    num_without_snapshots=100)
optimizer = torch.optim.Adam(pinn_net.parameters(), lr=0.001)
scheduler = None

pinn_trainer = Training.PINNTrainer(
    pinn_net, data, pinn_loss, optimizer, scheduler,
    input_normalization_pinn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    pinn_net, data, pinn_trainer, optimizer
)

In [ ]:
pinn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(pinn_trainer, pinn_net)

#### 4.5.3 Train PDNN

Given a training set $X_{PDNN} = ((\boldsymbol{\mu}^{(1)}, \operatorname{HF}(\boldsymbol{\mu}^{(1)})), \dots, (\boldsymbol{\mu}^{(n)}, \operatorname{HF}(\boldsymbol{\mu}^{(n)})))$ of parameter and high fidelity solution pairs for the PDE, we train a Projection-Driven Neural Network (PDNN) $\operatorname{N}_W(\boldsymbol{\mu})$ dependent on the weights and biases $W$ of the network to minimize the loss function
$$L_{PDNN}(X_{PDNN}; W) = \frac1n \sum_{i=1}^n \|\operatorname{N}_W(\boldsymbol{\mu}^{(i)}) - \tilde{\operatorname{HF}}(\boldsymbol{\mu}^{(i)})\|_2^2,$$
where for a given $\boldsymbol{\mu}$, $\tilde{\operatorname{HF}}(\boldsymbol{\mu})$ is the projection of $\operatorname{HF}(\boldsymbol{\mu})$ onto the reduced order solution space.

In [ ]:
input_normalization_pdnn = Normalization.StandardNormalization(input_normalization=True)
output_normalization_pdnn = Normalization.StandardNormalization()

pdnn_net  = NN.RONN("PDNN", problem, reduction_method, n_hidden=2, n_neurons=40)
pdnn_loss = Losses.PDNN_Loss(pdnn_net, output_normalization_pdnn)
data      = RONNData.RONNDataLoader(pdnn_net, validation_proportion=0.2)
optimizer = torch.optim.Adam(pdnn_net.parameters(), lr=0.001)

pdnn_trainer = Training.PDNNTrainer(
    pdnn_net, data, pdnn_loss, optimizer,
    input_normalization_pdnn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    pdnn_net, data, pdnn_trainer, optimizer
)

In [ ]:
pdnn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(pdnn_trainer, pdnn_net)

#### 4.5.4 Train PRNN

We train a Physics-Reinforced Neural Network (PRNN) $N_W(\boldsymbol{\mu})$ dependnent on the weights and biases $W$ of the network to minimize the loss function

$$L_{PRNN}(X_{PINN}, X_{PDNN}; W) = L_{PINN}(X_{PINN}; W) + \omega L_{PDNN}(X_{PDNN}; W)$$

where $\omega$ is a scaling parameter which can be chosen freely.

In [ ]:
input_normalization_prnn = Normalization.StandardNormalization(input_normalization=True)
output_normalization_prnn = Normalization.StandardNormalization()

omega = 1.
prnn_net  = NN.RONN(f"PRNN_{omega}", problem, reduction_method, n_hidden=2, n_neurons=40)
prnn_loss = Losses.PRNN_Loss(prnn_net, output_normalization_prnn, omega=omega)
data      = RONNData.RONNDataLoader(prnn_net, validation_proportion=0.2,
                                    num_without_snapshots=100)
optimizer = torch.optim.Adam(prnn_net.parameters(), lr=0.001)

prnn_trainer = Training.PRNNTrainer(
    prnn_net, data, prnn_loss, optimizer,
    input_normalization_prnn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    prnn_net, data, prnn_trainer, optimizer
)

In [ ]:
prnn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(prnn_trainer, prnn_net, separate=True)

### 4.6. Perform an error analysis

#### 4.6.1 Reduction Method Error Analysis

In [ ]:
reduction_method.initialize_testing_set(50, DEIM=60)
#reduction_method.error_analysis(filename="error_analysis")

#### 4.6.2 PINN Error Analysis

In [ ]:
test_mu = torch.tensor(reduction_method.testing_set)

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    pinn_net, test_mu, input_normalization_pinn, output_normalization_pinn, relative=True
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    pinn_net, (0.3, -1.0), input_normalization_pinn, output_normalization_pinn
)

#### 4.6.3 PDNN Error Analysis

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    pdnn_net, test_mu, input_normalization_pdnn, output_normalization_pdnn
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    pdnn_net, (0.3, -1.0), input_normalization_pdnn, output_normalization_pdnn
)

#### 4.6.4 PRNN Error Analysis

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    prnn_net, test_mu, input_normalization_prnn, output_normalization_prnn
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    prnn_net, (0.3, -1.0), input_normalization_prnn, output_normalization_prnn
)

#### 4.6.5 Neural Network Error Comparison

In [ ]:
nets = dict()
nets["pinn_net"] = pinn_net
nets["pdnn_net"] = pdnn_net
nets["prnn_net"] = prnn_net

input_normalizations = dict()
input_normalizations["pinn_net"] = input_normalization_pinn
input_normalizations["pdnn_net"] = input_normalization_pdnn
input_normalizations["prnn_net"] = input_normalization_prnn

output_normalizations = dict()
output_normalizations["pinn_net"] = output_normalization_pinn
output_normalizations["pdnn_net"] = output_normalization_pdnn
output_normalizations["prnn_net"] = output_normalization_prnn

_ = ErrorAnalysis.error_analysis_by_network(
    nets, test_mu, input_normalizations, output_normalizations, euclidean=False
)